In [57]:
import os
import time

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

SAVING_DIR='/home/data/taxonomy/'
os.environ["TRANSFORMERS_CACHE"] = SAVING_DIR + "hf_cache/"
os.environ["HF_HOME"] = SAVING_DIR + "hf_cache/"

import torch
from transformers import AutoModelForCausalLM, LlamaForCausalLM, LlamaTokenizer, AutoConfig, PretrainedConfig, AutoTokenizer
from peft import PeftModel 
import math
import re
from operator import attrgetter
import torch.nn as nn
import torch.nn.functional as F
import transformers

from typing import Optional, Dict, Sequence
from argparse import ArgumentParser
from pathlib import Path

from tqdm import tqdm
from dataclasses import dataclass, field
from itertools import chain

import numpy as np
import torch
from tqdm import tqdm
from peft import PeftModel, get_peft_model, TaskType, LoraConfig

from ste_utils import prepare_llama_ste
import datasets
from datasets import load_dataset

#import transformers
from transformers_modified.src.transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    default_data_collator,
    LlamaForCausalLM
)
import transformers_modified.src.transformers as transformers


IGNORE_INDEX = -100

In [68]:

@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """
    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "The model checkpoint for weights initialization. Don't set if you want to train a model from scratch."
            )
        },
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=False,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    token: str = field(
        default=None,
        metadata={
            "help": (
                "The token to use as HTTP bearer authorization for remote files. If not specified, will use the token "
                "generated when running `huggingface-cli login` (stored in `~/.huggingface`)."
            )
        },
    )
    trust_remote_code: bool = field(
        default=False,
        metadata={
            "help": (
                "Whether or not to allow for custom models defined on the Hub in their own modeling files. This option"
                "should only be set to `True` for repositories you trust and in which you have read the code, as it will "
                "execute code present on the Hub on your local machine."
            )
        },
    )
    torch_dtype: Optional[str] = field(
        default=None,
        metadata={
            "help": (
                "Override the default `torch.dtype` and load the model under this dtype. If `auto` is passed, the "
                "dtype will be automatically derived from the model's weights."
            ),
            "choices": ["auto", "bfloat16", "float16", "float32"],
        },
    )

@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    streaming: bool = field(default=False, metadata={"help": "Enable streaming mode"})
    block_size: Optional[int] = field(
        default=None,
        metadata={
            "help": (
                "Optional input sequence length after tokenization. "
                "The training dataset will be truncated in block of this size for training. "
                "Default to the model max input length for single sentence inputs (take into account special tokens)."
            )
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    validation_split_percentage: Optional[int] = field(
        default=5,
        metadata={
            "help": "The percentage of the train set used as validation set in case there's no validation split"
        },
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )

    dataset_percentage: Optional[int] = field(
        default=100,
        metadata={"help": "The number of percentage to take from entire dataset"},
    )


def load_hf_datasets(
    data_args
):
    # Load the dataset
    if data_args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        raw_datasets = load_dataset(
            data_args.dataset_name,
            data_args.dataset_config_name,
            streaming=data_args.streaming,
        )
        # if "validation" not in raw_datasets.keys():
        #     raw_datasets["validation"] = load_dataset(
        #         data_args.dataset_name,
        #         data_args.dataset_config_name,
        #         split=f"train[:{data_args.validation_split_percentage}%]",
        #         streaming=data_args.streaming,
        #     )
        #     raw_datasets["train"] = load_dataset(
        #         data_args.dataset_name,
        #         data_args.dataset_config_name,
        #         split=f"train[{data_args.validation_split_percentage}%:]",
        #         streaming=data_args.streaming,
        #     )

    if data_args.dataset_percentage < 100: 
        dataset_frac = data_args.dataset_percentage/100
        dataset_parts = raw_datasets['train'].train_test_split(train_size=dataset_frac)
        raw_datasets['train'] = dataset_parts['train']
      #  dataset_parts = raw_datasets['validation'].train_test_split(test_size=dataset_frac)
      #  raw_datasets['validation'] = dataset_parts['test']
        
    return raw_datasets

def tokenize_datasets(data_args, raw_datasets, tokenizer):
    dataset_type = list(raw_datasets.keys())[0]
    column_names = list(raw_datasets[dataset_type].features)
    text_column_name = "text" if "text" in column_names else column_names[0]

    def tokenize_function(examples):
        output = tokenizer(examples[text_column_name])
        return output

    if not data_args.streaming:
        tokenized_datasets = raw_datasets.map(
            tokenize_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on dataset",
        )
    else:
        tokenized_datasets = raw_datasets.map(
            tokenize_function,
            batched=True,
            remove_columns=column_names,
        )

    return tokenized_datasets

def format_datasets(
    data_args,
    tokenized_datasets,
    tokenizer
):
    
    block_size = min(data_args.block_size, tokenizer.model_max_length)
    print(block_size)

    # Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
    def group_texts(
        examples
    ):
        # Concatenate all texts.
        concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        # We drop the small remainder, and if the total_length < block_size  we exclude this batch and return an empty dict.
        # We could add padding if the model supported it instead of this drop, you can customize this part to your needs.
        total_length = (total_length // block_size) * block_size
        # Split by chunks of max_len.
        result = {
            k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        
        return result


    if not data_args.streaming:
        lm_datasets = tokenized_datasets.map(
            group_texts,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            load_from_cache_file=not data_args.overwrite_cache,
            desc=f"Grouping texts in chunks of {block_size}",
        )
    else:
        lm_datasets = tokenized_datasets.map(
            group_texts,
            batched=True,
        )
    
    return lm_datasets

@dataclass
class DataCollatorWithMaskForCausalLM(object):
    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, batch):
        input_ids = []
        labels = []
        attention_masks = []

        for item_dict in batch:
            input_ids.append(torch.tensor(item_dict["input_ids"]))
            attention_masks.append(torch.tensor(item_dict["attention_mask"]))
            label = torch.tensor(item_dict["labels"])
            label[:-1] = IGNORE_INDEX
            labels.append(label)

        input_ids = torch.vstack(input_ids)
        attention_masks = torch.vstack(attention_masks)
        labels = torch.vstack(labels)
            
        data_dict = {
            'input_ids': input_ids,
            'attention_mask': attention_masks,
        }
        if labels is not None:
            data_dict['labels'] = labels
        return data_dict
    



In [63]:
dataset_name = '/home/LLM_Compression/logs/wikitext_gpt2'
#dataset_config_name = 'wikitext-2-raw-v1'
valid_split = 1
block_size = 32
dataset_percentage = 100

data_args = DataTrainingArguments(
    dataset_name = dataset_name,
    dataset_config_name = dataset_config_name,
    validation_split_percentage = valid_split,
    block_size = block_size,
    dataset_percentage = dataset_percentage
)


In [60]:
model_name_or_path = 'gpt2'
cache_dir = SAVING_DIR + "hf_cache/"

model_args = ModelArguments(
    model_name_or_path = model_name_or_path,
    config_name = None, 
    tokenizer_name = None,
    use_fast_tokenizer = True,
    token = None,
    trust_remote_code = True,
    cache_dir= cache_dir
)


In [73]:
 model = AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        torch_dtype=torch.bfloat16,
        cache_dir=model_args.cache_dir,
        device_map="auto"	    
)

In [61]:

# Load pretrained tokenizer
tokenizer_kwargs = {
    "cache_dir": model_args.cache_dir,
    "use_fast": model_args.use_fast_tokenizer,
    "revision": model_args.model_revision,
    "token": model_args.token,
    "trust_remote_code": model_args.trust_remote_code,
}

if model_args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(model_args.tokenizer_name, **tokenizer_kwargs)
elif model_args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, **tokenizer_kwargs)

In [69]:
raw_datasets = load_hf_datasets(data_args)

Resolving data files: 100%|██████████| 20/20 [00:00<00:00, 49725.00it/s]


Resolving data files: 100%|██████████| 229/229 [00:00<00:00, 113695.03it/s]


In [71]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'logits'],
        num_rows: 734
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'logits'],
        num_rows: 66
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'logits'],
        num_rows: 8850
    })
})

In [74]:
# tokenized_datasets = tokenize_datasets(data_args, raw_datasets, tokenizer)
# lm_datasets = format_datasets(data_args, tokenized_datasets, tokenizer)

data_collator = DataCollatorWithMaskForCausalLM(tokenizer=tokenizer)

In [12]:
lm_datasets['train'] = lm_datasets['train'].add_column('logits', [None]*len(lm_datasets['train']))

In [50]:
lm_datasets.pop('test')

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 8850
})

In [51]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 45
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7
    })
})

In [35]:
def format_logit(example):
    logits = model(torch.tensor(example['input_ids']).to(model.device), attention_mask=torch.tensor(example['attention_mask']).to(model.device)).logits
    example['logits'] = logits.cpu().tolist()
    return example

In [42]:
ld = lm_datasets.map(format_logit, batched=True, batch_size=2)

Map:   0%|          | 0/78 [00:00<?, ? examples/s]

Map: 100%|██████████| 4/4 [00:01<00:00,  3.77 examples/s]


In [43]:
ld

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'logits'],
        num_rows: 78
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'logits'],
        num_rows: 4
    })
})

In [44]:
ld.save_to_disk('logs/test_dataset')

Saving the dataset (1/1 shards): 100%|██████████| 4/4 [00:00<00:00, 27.26 examples/s]


In [75]:
lm_datasets = raw_datasets


training_args = TrainingArguments(
    output_dir = './test_distill',
    learning_rate = 3e-4, 
    seed = 2, 
    num_train_epochs = 1,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    gradient_accumulation_steps = 1,
    gradient_checkpointing=False,
    save_strategy = 'steps',
    save_steps = 1000,
    evaluation_strategy = 'steps',
    eval_steps = 10,
    weight_decay = 0.1,
    warmup_ratio = 0.03,
    lr_scheduler_type = "cosine",
    logging_steps = 1,
    do_train = True,
    do_eval = False,
)

train_dataset = lm_datasets["train"]
eval_dataset = lm_datasets["validation"]

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    # Data collator will default to DataCollatorWithPadding, so we change it.
    data_collator=data_collator
)

In [77]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: vityavitalich. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
10,5.656200,4.674124
20,8.875000,4.738814
30,2.906200,4.796757
40,7.750000,4.627604
50,4.968800,4.710819


KeyboardInterrupt: 

In [13]:
outputs = trainer.predict(eval_dataset)

In [17]:
outputs.predictions.shape

(54, 32, 50257)

In [19]:
logits = torch.rand(5000, 1024, 50257)

In [ ]:
raw_datase

In [97]:
outputs.predictions[0].shape

(32, 50257)

In [98]:
out = model.forward(torch.tensor([[0, 1]]))

In [101]:
out.logits.shape

torch.Size([1, 2, 50257])

In [63]:
logits[0][0].shape

(12, 8, 32, 64)

In [45]:
logits[0][0].shape

(12, 8, 32, 64)

In [49]:
len(eval_dataset[0]['input_ids'])

32

In [53]:
eval_dataset[1]

{'input_ids': [247,
  8230,
  281,
  253,
  2798,
  273,
  27015,
  41151,
  23749,
  1157,
  253,
  2846,
  1907,
  644,
  387,
  253,
  43213,
  273,
  1302,
  314,
  1495,
  1157,
  1445,
  285,
  10336,
  323,
  689,
  247,
  8014,
  1107,
  1157,
  342],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [247,
  8230,
  281,
  253,
  2798,
  273,
  27015,
  41151,
  23749,
  1157,
  253,
  2846,
  1907,
  644,
  387,
  253,
  43213,
  273,
  1302,
  314,
  1495,
  1157,
  1445,
  285,
  10336,
  323,
  689,
  247,
  8014,
  1107,
  1157,
  342]}

In [72]:
logits[1][1].shape

(12, 8, 32, 64)

In [74]:
64 * 8 * 12

6144

In [73]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [ ]:

def run_train(
    model_args,
    data_args,
    training_args,
    config,
):
    
    # Load pretrained model
    # if config.model_type == 'Llama':
    #     model_type = LlamaForCausalLM
    # else:
    #     model_type = AutoModelForCausalLM
    

    model = AutoModelForCausalLM.from_pretrained(
        model_args.model_name_or_path,
        torch_dtype=torch.bfloat16,
        token=model_args.token,
        cache_dir=model_args.cache_dir,
        device_map="auto"	    
)
    
    if config.zero_outliers:
        make_zero_outliers(model, config.outlier_fraction)

    if config.use_clip_softmax:
        model.set_clipped_sm(gamma=config.clip_softmax_gamma, eta=config.clip_softmax_eta)

    if config.ste.enable:
        outlier_ids, layer_bit = prepare_llama_ste(config.ste.path_to_act_scales, config.ste.fp_features_num, **config.ste.layer_bits)
        model.enable_ste(outlier_ids=outlier_ids, layer_bit=layer_bit, block_size=config.ste.block_size)

    if config.use_lora:
        task_type = TaskType.CAUSAL_LM
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "down_proj", "gate_proj"]
        lora_config = LoraConfig(
            task_type=task_type,
            inference_mode=False,
            r=config.lora_rank,
            lora_alpha=config.lora_alpha,
            lora_dropout=config.lora_dropout,
            target_modules=config.lora_target_modules,
            init_lora_weights=True,
        )
        model = get_peft_model(model, lora_config)

    # Load pretrained tokenizer
    tokenizer_kwargs = {
        "cache_dir": model_args.cache_dir,
        "use_fast": model_args.use_fast_tokenizer,
        "revision": model_args.model_revision,
        "token": model_args.token,
        "trust_remote_code": model_args.trust_remote_code,
    }

    if model_args.tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(model_args.tokenizer_name, **tokenizer_kwargs)
    elif model_args.model_name_or_path:
        tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path, **tokenizer_kwargs)

    #Load and preprocessing dataset
    raw_datasets = load_hf_datasets(data_args)
    tokenized_datasets = tokenize_datasets(data_args, raw_datasets, tokenizer)
    lm_datasets = format_datasets(data_args, tokenized_datasets, tokenizer)

    data_collator = DataCollatorWithMaskForCausalLM(
        tokenizer=tokenizer
    )

    if config.norm_tweek:
        layernorm_names = [f"model.layers.{layer_block_num}.input_layernorm.weight" for layer_block_num in range(len(model.model.layers))]
        layernorm_names += [f"model.layers.{layer_block_num}.post_attention_layernorm.weight" for layer_block_num in range(len(model.model.layers))]

        #Set model parameters to be learned
        for name, param in model.named_parameters():
            if name not in layernorm_names:
                # freeze base model's layers
                param.requires_grad = False
            else:
                param.requires_grad = True

    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(f"trainable_params: {trainable_params}")

    
    #Train
    train_dataset = lm_datasets["train"]
    eval_dataset = lm_datasets["validation"]

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        # Data collator will default to DataCollatorWithPadding, so we change it.
        data_collator=default_data_collator
    )

    trainer.save_model()
    train_result = trainer.train()
    trainer.save_model()  # Saves the tokenizer too for easy upload

